In [ ]:
from code.model.main import End_to_end_neural_el, preprocessing, prepro_util_main

print ("import done")

In [ ]:
#Object creation

end_to_end_el_obj = End_to_end_neural_el()

In [ ]:
# Pre processing data
input_file_path = "../data/basic_data/test_datasets/AIDA/"
output_file_path = "../data/new_datasets/"
dataset_name = "AIDA"
preprocessing(dataset_name, input_folder, output_folder)
prepro_util_main()

print ("preprocessing done")

In [ ]:
# Read the preprocessed data
# Prepare Arguments for training - These were  read from command line in the original implementation
args_dict = {}
args_dict['batch_size'] = 4
args_dict['experiment_name'] = "paper_models"
args_dict['training_name'] = "group_global / global_model_v$v"
args_dict['ent_vecs_regularization'] = "l2dropout"
args_dict['evaluation_minutes'] = 10
args_dict['nepoch_no_imprv'] = 6
args_dict['span_emb'] = "boundaries"
args_dict['dim_char'] = 50
args_dict['hidden_size_char'] = 50
args_dict['hidden_size_lstm'] = 150
args_dict['nn_components'] = "pem_lstm_attention_global"
args_dict['fast_evaluation'] = True
args_dict['all_spans_training'] = True
args_dict['attention_ent_vecs_no_regularization'] = True
args_dict['final_score_ffnn'] = "0_0"
args_dict['attention_R'] = 10
args_dict['attention_K'] = 100
args_dict['train_datasets'] = "aida_train"
args_dict['el_datasets'] = "aida_dev_z_aida_test_z_aida_train"
args_dict['el_val_datasets'] = 0
args_dict['global_thr'] = 0.001
args_dict['global_score_ffnn'] = "0_0"

args = _parse_args_train(args_dict)

# Read data for EL Task

folder = "../data/tfrecords/"+args_dict['experiment_name']+\
                ("/allspans/" if args.all_spans_training else "/gmonly/")
dataset_path = [folder + file for file in args.train_datasets]
training_dataset = end_to_end_el_obj.read_dataset(dataset_path, args_dict)

print ("Reading for EL done")

In [ ]:
# Training for EL task
trained_model_ed = end_to_end_el_obj.train([training_dataset])

print ("Training for EL done")

In [ ]:
# Read data for ED Task
args_dict['all_spans_training'] = False
args = _parse_args_train(args_dict)

folder = "../data/tfrecords/" + args_dict['experiment_name'] + \
             ("/allspans/" if args.all_spans_training else "/gmonly/")
dataset_path = [folder + file for file in args.train_datasets]
training_dataset = end_to_end_el_obj.read_dataset(dataset_path, args_dict)

print ("Reading for ED done")

In [ ]:
# Training for ED task
trained_model_ed = end_to_end_el_obj.train([training_dataset])

print ("Training for ED done")

In [ ]:
# Predictions
prediction_datasets = ["aida_train.txt_z_aida_dev.txt_z_aida_test.txt"]  # <-- as we need list input
predictions = end_to_end_el_obj.predict(trained_model_el, prediction_datasets)

print ("Predicting done")

In [ ]:
# Evaluations
base_folder = "../../data/tfrecords/"
dev_set = "aida_dev.txt"
test_set = "aida_test.txt"
required_data = [base_folder, dev_set, test_set]
precision, recall, F1 = end_to_end_el_obj.evaluate(trained_model_el, required_data)

print ("Accuracy :", precision, recall, F1)